In [1]:
!pip install transformers bitsandbytes peft accelerate datasets huggingface_hub
!pip install langchain langsmith openai langchain-openai langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import langsmith

from google.colab import userdata
key = userdata.get('Openai')
lc_key = userdata.get('langsmith')
os.environ['OPENAI_API_KEY']=key

os.environ['LANGCHAIN_TRACING_V2']='true'
os.environ['LANGCHAIN_API_KEY']= lc_key


os.environ['LANGCHAIN_PROJECT']='Rude_RAG'

In [3]:
from huggingface_hub import login
hg_token = userdata.get('huggingface')
login(token=hg_token)

In [4]:
import transformers
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
model_id = 'NotoriousH2/rude_rag'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map={'':0})

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,
                max_new_tokens=300,return_full_text=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/37.0M [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
import csv

In [10]:
llm = HuggingFacePipeline(pipeline=pipe)

chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer)

In [29]:
csv_file_path = '/content/drive/MyDrive/fastcam/Part 12. LLM 활용 심화/실습 자료/RAG_Data_val.csv'

prompts = []
answers = []

with open(csv_file_path, 'r', encoding='utf-8') as file:
  reader = csv.DictReader(file)

  for row in reader:
    prompts.append({'context': row['context'],
                    'question': row['question']})
    answers.append({'answer':row['answer']})

print('exam_prompts:', prompts[0])
print('exam_answrs:', answers[0])

exam_prompts: {'context': '석굴암(石窟庵)은 대한민국 경상북도 경주시의 토함산 중턱(진현동 891)에 있는 불국사 소속 호국암자이다.\n국보 24호인 경주 석굴암 석굴이 있는 암자이다. 석굴암 석굴은 세계에서 유일한 인위적으로 만들어진 화강암 석굴이다. 내부에서는 보존을 위해 사진 촬영은 금지되어 있다.\n2023년 5월 4일부터 무료입장이 가능해졌다.\n\n\n== 개요 ==\n보통 석굴암의 암(庵)자를 바위 암(岩)자로 알고. 석굴암 석굴을 동일한 용어로 쓴다. 하지만 석굴암의 암(庵)자는 암자 암(庵)자로. 석굴이 있는 암자(작은 절)를 뜻한다. 석굴암 석굴은 불상과 부속 화강암 조각이 있는 굴을 뜻한다.\n남북국시대 751년(통일신라 경덕왕 10년)에 김대성이 만들었을 때는 석불사(石佛寺)라고 하였다. 큰 규모의 절을 한 글자로 사. 작은 규모의 절은 한 글자로 암이라고 부르므로. 창건 당시에 석굴암은 규모가 현재보다 컸던 걸로 생각된다.\n\n\n== 창건 및 연혁 ==\n『불국사고금창기(佛國寺古今創記)』를 따른다.\n\n\n=== 남북국 시대 ===\n751년(통일신라 경덕왕 10년) : 재상 김대성이 석불사(石佛寺)로 창건을 시작하였다.\n774년(통일신라 혜공왕 10년) : 김대성이 석굴암(석불사)을 완공하였다.\n\n\n=== 조선 시대 ===\n1703년(숙종 29년) : 종열(從悅)이 중수하였다.\n1758년(영조 34년) : 대겸(大謙)이 중수하였다.\n조선 말기 : 울산병사 조예상(趙禮相)이 중수하였다.\n\n\n=== 구한말 및 일제강점기 ===\n1907년 : 1962년 지역 노인들의 증언에 따르면. 이때 석굴암은 조가절(趙家寺)이라고 불리며 일반인들이 향을 올리고 공양을 계속하고 있었다. 그러나 우체부가 석굴을 새로 발견한듯이 우체국장에게 보고 하였다. 일제도 이에 동조하여. 일본인들이 석굴을 훼손하고 문화재를 반출해가는 계기로 만들어 버렸다.\n\n\n== 창건 이유 ==\n\n\n=== 전통적으로 알려진 사

# 데이터셋 평가

In [32]:
ls_client = langsmith.Client()
dataset_name = 'Rude RAG Validation2'

dataset = Is_client.create_dataset(dataset_name)
ls_client.create_examples(
    inputs=prompts,
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['2c4b8cae-46a4-4ea4-b4c9-8ed0028bf714',
  'f07d7c42-bd0f-4407-bf73-44baa1853bf7',
  'a68606b4-1797-4e16-8c19-8ebfc780e130',
  '15e3e979-57d5-4b4b-bad2-5cb7583d7b00',
  '4193dd97-376c-464f-8d16-8540e43e5747',
  'b99d5142-111d-4775-83b1-02435c94c910',
  '4b71cb4e-73c7-406d-9305-6130e7b463ae',
  '2e882da9-d4a8-4e65-827a-d91669ccd829',
  'ef840713-2273-421c-91da-3fb475ed80b6',
  '31ba8eb1-1021-42c5-8e62-471baa736984',
  '621bf51f-367b-4906-9466-35ab71845cc6',
  '4b78a6a8-cfee-47b7-95c8-70bc20e67e6b',
  'f597db48-19d6-4a1d-8759-5eab200954d7',
  '59453908-3546-4d1b-b52e-35e5a1fbd6d6',
  'ba73c6ed-e593-4071-9407-fd27694fdffe',
  'd73c0dd7-2610-4534-9ba6-b291f3a73fdf',
  'e088c5d5-e994-45ef-ba1a-de01c2cb6d2e',
  '83bc03d7-2af6-4a92-8359-8a87db56b90c',
  '28ed9528-7181-4d49-b837-90c63780f7e3',
  '16c0432c-8626-4d96-a2fb-c3be40403cdd',
  '554b80fa-30df-410d-9c46-a684483b1422',
  'ab8c44e7-4d65-478b-8ce8-23c8a8ecd68a',
  '2ea23d33-ee0b-49dc-b16d-0e07923b197a',
  'c12a7548-c463-4f

## 평가 프롬프트

In [37]:
from langsmith.schemas import Run, Example

_PROMPT_TEMPLATE = """당신은 질문에 대해 제출된 응시자의 답안과 정답을 비교하여
평가하는 전문가입니다.
질문은 아래와 같습니다:
{query}

정답은 아래와 같습니다:
{answer}

제출된 답안은 아래와 같습니다:
Submitted {result}

평가 기준은 다음과 같습니다.
각각을 100점 만점으로 평가한 뒤, 평균 점수를 Grade에 계산하세요.
Q1. 답안의 말투가 정답과 유사한가? (5점 단위)
Q2. 답안과 정답이 유사한 결론을 내리고 있는가? (5점 단위)
Q3. 답안과 정답이 유사한 근거를 대고 있는가? (5점 단위)
Grade: 평균 점수

Q1, Q2, Q3, Grade를 Key로 하는 Json 형식으로 작성하세요.
다른 내용은 출력하지 마세요.
---
Q1:
Q2:
Q3:
Grade:
"""

In [38]:
PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)
eval_llm = ChatOpenAI(model='gpt-4o',temperature=0.0)

In [39]:
def evaluate_answer(run: Run, example: Example) -> dict:
    evaluate_chain = PROMPT | eval_llm | JsonOutputParser()

    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("answer") or ""
    question = example.inputs.get("question") or ""

    score = evaluate_chain.invoke({"query": question, "answer": required, "result": prediction})
    return {'results':[
        {'key':'Average',
        'score':float(score['Grade'])/100},

        {'key':'Style',
        'score':float(score['Q1'])/100},

        {'key':'Correctness',
        'score':float(score['Q2'])/100},

        {'key':'Reliability',
        'score':float(score['Q3'])/100}]}


In [41]:
chat_prompt = ChatPromptTemplate.from_messages([
    ('user','''Context: {context}

---

{question}''')
])

RAG_chain = chat_prompt | chat_model | StrOutputParser()

In [42]:
def rude_rag(data):
  return {'output':RAG_chain.invoke(data)}

In [36]:
from langsmith.evaluation import evaluate

experiment_results = evaluate(
    rude_rag,
    data = dataset_name,
    evaluators = [evaluate_answer],
    experiment_prefix = 'openai-4o-QWEN2-1.5B-instruct'
)

View the evaluation results for experiment: 'openai-4o-QWEN2-1.5B-instruct-37a6e89a' at:
https://smith.langchain.com/o/e2d407dd-f01a-47a5-8377-c5491b1ba2ad/datasets/308b61c7-55ec-4510-8c55-e165c5983aca/compare?selectedSessions=63f8e82c-fe86-4982-9c7c-a7d83c5be967




0it [00:00, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


KeyboardInterrupt: 